### Objective: 

for a given gene, get the set of N nearest neighbors in UMAP space. 

### Imports: 

In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

### Files: 

In [2]:
# Review how you generated this file:
file_umap = '../../data/standardized_data/df_umap_lfc_08_08_24.csv'

### Functions: 

In [3]:
def get_neighbors(data, n_neighbors = 5, ix = None, **kwargs): 
    """
    Given an input dataframe with a given coordinate system, 
    the function returns the indices of the nearest neighbors
    of a dataset or a specific data point. 

    Params 
    ------
    data (pd.DataFrame)
        Dataframe whose columns represent a given coordinate system where distances
        can be calculated.

    n_neighbors(int, default = 5)
        Number of nearest neighbors. 
        
    ix (int, default = None)
        Index of data point for which to find the nearest neighbors.
        Defaults to None, to return the indices of all data points. 

    Returns 
    -------
    neighbor_ixs (np.array)
        Array of the indices of the nearest neighbors of shape (n_datapoints, n_neighbors).
    """

    # Initalize Nearest Neighbors object
    knn = NearestNeighbors(n_neighbors = n_neighbors).fit(data)

    neighbors = knn.kneighbors()

    neighbor_ixs = neighbors[1]

    # Loop to return ixs of a specific point.
    if ix is not None: 
        return neighbor_ixs[ix]

    return neighbor_ixs


def get_neighbor_subdataset(neighbors, data, col_name, id_): 

    """
    Using the precomputed neighbor indices, returns a subsdataset 
    for an input datapoint with index id. 

    Params 
    ------

    neighbors (np.array)
        Neighbor indices of size (n_datapoints, n_neighbors). 

    data (pd.DataFrame)
        Pandas dataframe to be filtered.

    col_name (str)
        Column name for filtering. 

    id_ (str)
        Identifier to use for getting 
    
    Returns 
    -------
    sub_df (pd.DataFrame)
        Subset of data of the point's neighbors.
    """

    # Extract index of desired data point 
    target_ix = data[data[col_name].values == id_].index.values[0]

    # Get the neighbors indices of desired data point
    neighbor_ixs = neighbors[target_ix]
    
    # Get subdataset using iloc method
    sub_df = data.iloc[neighbor_ixs]
    
    return sub_df 

### Example: 

load UMAP dataframe: 

In [4]:
df_lfc_annot = pd.read_csv(file_umap)
df_lfc_annot = df_lfc_annot[['Rv_ID', 'gene_name', 'Annotation', 'func_tuberculist', 'u1', 'u2']]
df_lfc_annot.head(3)

,Rv_ID,gene_name,Annotation,func_tuberculist,u1,u2
0,Rv0001,dnaA,4.0,information pathways,2.411358,-7.368723
1,Rv0002,dnaN,4.0,information pathways,2.998742,-7.274495
2,Rv0003,recF,2.0,information pathways,1.793634,-5.717894


This function pre-computes N nearest neighbors for all genes and stores in a list of index lists.

In [5]:
#neighbors = get_neighbors( df_lfc_annot[['u1', 'u2', 'u3']], n_neighbors = 60, ix = None )
neighbors = get_neighbors( df_lfc_annot[['u1', 'u2']], n_neighbors = 60, ix = None )

In [6]:
neighbors

array([[ 707, 2439,  839, ..., 3087, 2642, 2421],
       [4039, 3719, 3004, ...,  706, 1500, 3001],
       [4054, 1565, 2829, ..., 1915,  590, 3490],
       ...,
       [1111, 3083, 3084, ..., 1688, 4050, 1686],
       [2917, 1925, 2055, ..., 2586, 3012, 3115],
       [3090, 4039,    1, ..., 2769, 1040, 2541]])

And here we select the N nearest neighbors for a given query gene: 

In [7]:
rvid = 'Rv3502c'
df_subset = get_neighbor_subdataset(neighbors, df_lfc_annot, col_name = 'Rv_ID', id_ = rvid)

If we only want to display the ones with high uniprot annotation score, we could do this: 

In [8]:
df_subset[df_subset.Annotation.values>=3]

,Rv_ID,gene_name,Annotation,func_tuberculist,u1,u2
4005,Rv3873,PPE68,4.0,PE/PPE,-1.342324,-2.459687
498,Rv0485,-,3.0,regulatory proteins,-1.032274,-2.591314
3980,Rv3849,espR,5.0,regulatory proteins,-1.007166,-2.583089
2129,Rv2069,sigC,5.0,information pathways,-1.015984,-2.613487
3665,Rv3543c,fadE29,5.0,lipid metabolism,-1.004491,-2.627719
4000,Rv3868,-,5.0,cell wall and cell processes,-0.929452,-2.617738
784,Rv0757,phoP,4.0,regulatory proteins,-1.616701,-2.351876
3692,Rv3569c,bphD,5.0,intermediary metabolism and respiration,-0.916831,-2.638451
3738,Rv3615c,-,4.0,cell wall and cell processes,-1.630839,-2.334867
3739,Rv3616c,-,5.0,cell wall and cell processes,-1.635267,-2.334137


#### Reformat data for Anisha's dash app:

In [9]:
# initialize NN dataframe: 
cols = ['Rv_ID']+['NN_'+str(i) for i in range(1, 11)]
df_NN_dash = pd.DataFrame(columns=cols)

list_rvid = df_lfc_annot.Rv_ID.to_list()
counter = 0
for rvid in list_rvid:
    if counter%250==0:
        print(counter, 'out of', len(list_rvid))
    # get neighbors: 
    df_NN = get_neighbor_subdataset(neighbors, df_lfc_annot, col_name = 'Rv_ID', id_ = rvid)
    df_NN_annot = df_NN[df_NN.Annotation.values>=3].copy()
    row_to_append = [rvid]+df_NN_annot.iloc[:10, 0].to_list() # 10 annotated NN's
    
    if len(row_to_append)<11:
        size_pad = 11 - len(row_to_append)
        row_to_append = row_to_append + size_pad*['NONE']
    
    df_NN_dash.loc[len(df_NN_dash)] = row_to_append
    counter += 1

0 out of 4058
250 out of 4058
500 out of 4058
750 out of 4058
1000 out of 4058
1250 out of 4058
1500 out of 4058
1750 out of 4058
2000 out of 4058
2250 out of 4058
2500 out of 4058
2750 out of 4058
3000 out of 4058
3250 out of 4058
3500 out of 4058
3750 out of 4058
4000 out of 4058


In [10]:
file_out = '../../data/NN_for_dash.csv'
df_NN_dash.to_csv(file_out, index = False)

In [11]:
df_NN_dash[df_NN_dash.Rv_ID.values == 'Rv3400']

,Rv_ID,NN_1,NN_2,NN_3,NN_4,NN_5,NN_6,NN_7,NN_8,NN_9,NN_10
3522,Rv3400,Rv3873,Rv0757,Rv0485,Rv3615c,Rv3616c,Rv3870,Rv3882c,Rv3614c,Rv3871,Rv3877
